<a href="https://colab.research.google.com/github/niyobern/Google-Colab-notebooks/blob/main/Telemedecine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installing Packages

In [ ]:
!pip install torch torchvision

# 2. Defining datasets directory

In [ ]:
data_dir = '/content/drive/MyDrive/Telemedicine/datasets/'

# 3. Loading and transforming data

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
dataset.classes

['COVID', 'noncovid']

In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Storing dataloaders in a dictionary for easy lookup
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

# Dataset sizes
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import ResNet18_Weights

# pretrained ResNet model
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)



In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has the best accuracy so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
import copy
# Let's train for 25 epochs to see
num_epochs = 15

model = train_model(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs=num_epochs)


Epoch 0/14
----------
train Loss: 0.4708 Acc: 0.7723
val Loss: 0.3110 Acc: 0.8552
Epoch 1/14
----------
train Loss: 0.2713 Acc: 0.8891
val Loss: 0.2417 Acc: 0.8937
Epoch 2/14
----------
train Loss: 0.1526 Acc: 0.9457
val Loss: 0.1948 Acc: 0.9368
Epoch 3/14
----------
train Loss: 0.1033 Acc: 0.9646
val Loss: 0.1940 Acc: 0.9307
Epoch 4/14
----------
train Loss: 0.0838 Acc: 0.9723
val Loss: 0.2135 Acc: 0.9168
Epoch 5/14
----------
train Loss: 0.0564 Acc: 0.9788
val Loss: 0.1300 Acc: 0.9553
Epoch 6/14
----------
train Loss: 0.0348 Acc: 0.9904
val Loss: 0.1246 Acc: 0.9599
Epoch 7/14
----------
train Loss: 0.0227 Acc: 0.9958
val Loss: 0.1424 Acc: 0.9507
Epoch 8/14
----------
train Loss: 0.0222 Acc: 0.9938
val Loss: 0.1243 Acc: 0.9646
Epoch 9/14
----------
train Loss: 0.0146 Acc: 0.9973
val Loss: 0.1247 Acc: 0.9676
Epoch 10/14
----------
train Loss: 0.0137 Acc: 0.9973
val Loss: 0.1233 Acc: 0.9569
Epoch 11/14
----------
train Loss: 0.0519 Acc: 0.9838
val Loss: 0.1141 Acc: 0.9707
Epoch 12/14
--

In [ ]:
torch.save(model, '/content/drive/MyDrive/Telemedicine/telemedecine.pth')
